In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installation Steps

In [ ]:
#Installing PyTorch
!pip3 install --quiet torch torchvision torchaudio

#Install HuggingFace Transformer
!pip install --quiet transformers

#Install tqdm for progress bars
!pip install --quiet tqdm

#Essential for automodels where models are not directly from huggingface pre-trained libs
!pip install --quiet sentencepiece

#Install pytorch lightning
!pip install --quiet git+https://github.com/PyTorchLightning/pytorch-lightning

#Install tqdm for progress bar
!pip install --quiet tqdm

#Install pandas
!pip install --quiet pandas

#Install sklearn
!pip install --quiet sklearn

#Install termcolor
!pip install --quiet termcolor

#Install tensorflow
!pip install --quiet tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 KB 7.3 MB

In [ ]:
## Specially for Jupyterhub

In [ ]:
# for JupyterHub
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp38-cp38-linux_x86_64.whl (2041.3 MB)
ERROR: pytorch-lightning 1.9.0 has requirement torch>=1.10.0, but you'll have torch 1.9.0+cu111 which is incompatible.
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install --quiet pytorch-lightning==1.1.3

In [ ]:
!pip install --quiet --upgrade pytorch-lightning
!pip install --quiet --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.0 MB/s eta 0:00:00


# Verify Installations

In [ ]:
!sudo apt install python3

In [ ]:
#To test if HuggingFace is properly installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

# Load T5X Model

Initialize and retrieve model for training

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, Adafactor

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from termcolor import colored
from tqdm import tqdm


checkpoint = "google/t5-v1_1-large" #"google/t5-v1_1-xl"  #"google/t5-v1_1-xl" is too huge for google colab to handle

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


## Create Torch Dataset Class for our custom dataframe

In [ ]:
class ClaimDecompDataset(Dataset):

  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: AutoTokenizer,
      source_max_tok_len: int = 850,
      target_max_tok_len: int = 500
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_tok_len
    self.target_max_token_len = target_max_tok_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    sample_row = self.data.iloc[index]

    source_encoding = tokenizer(
        sample_row["claim"],
        sample_row["justification"],
        max_length = self.source_max_token_len,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    target_encoding = tokenizer(
        sample_row["questions"],
        max_length = self.target_max_token_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    labels = target_encoding["input_ids"]
    labels[labels == 0] = -100

    return dict(
      claim = sample_row["claim"],
      justification = sample_row["justification"],
      questions = sample_row["questions"],
      input_ids = source_encoding["input_ids"].flatten(),
      attention_mask = source_encoding["attention_mask"].flatten(),
      labels = labels.flatten()
    )

## Initialize Dataset

In [ ]:
train_dataset = ClaimDecompDataset(dfTrain, tokenizer)
dev_dataset = ClaimDecompDataset(dfDev, tokenizer)
test_dataset = ClaimDecompDataset(dfTest, tokenizer)

In [ ]:
dev_dataset.data.shape

(197, 3)

The following cell contains code not required in the final .py file

In [ ]:
#Preview data in custom constructed dataset
for data in dev_dataset:
  print(data["claim"])
  print("\n")
  print(data["justification"])
  print("\n")
  print(data["questions"])
  print("\n")
  print(data["input_ids"][:10])
  print(data["labels"][:10])
  print("\n")
  print(tokenizer.decode(data["labels"].squeeze()))
  break
  

Generate Questions: With voting by mail, “you get thousands and thousands of people sitting in somebody's living room, signing ballots all over the place.”


Justification: Trump said that with voting by mail, "you get thousands and thousands of people sitting in somebody's living room, signing ballots all over the place."
Voting fraud in general is considered to be rare, although voting experts agree that the risks are greater for mail balloting than for in-person voting. Still, Trump didn’t produce any evidence for the "thousands and thousands" claim, and voting experts said his assertion doesn’t square with what is known about the actual cases of voting fraud in the recent past.
We rate the statement False.


Is voting fraud widespread in the US? Is there a greater risk of voting fraud with mail-in ballots? Is there evidence of thousands of people committing mail-in voting fraud? Is mail balloting at great risk for fraudulent voting than in-person voting? Are there thousands of peop

OverflowError: out of range integral type conversion attempted

## Creating our own lightning module

#### Building our lightning module

In [ ]:
class ClaimDecompDataModule(pl.LightningDataModule):

  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer: AutoTokenizer,
      batch_size: int = 2,
      source_max_token_len: int = 850,
      target_max_token_len: int = 500
  ):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = ClaimDecompDataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.test_dataset = ClaimDecompDataset(
        self.test_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle = True,
        num_workers = 4
    )

  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = 1,
        num_workers = 4
    )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = 1,
        num_workers = 4
    )


#### Establish key parameters

In [ ]:
BATCH_SIZE = 1
N_EPOCHS = 7

data_module = ClaimDecompDataModule(dfTrain, dfDev, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

#### Build custom model in native Pytorch

In [ ]:
class ClaimDecompModel(pl.LightningModule):
  
  def __init__(self):
    super().__init__()
    self.model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-large")

  def forward(self, input_ids, attention_mask, labels = None):
    output = self.model(
        input_ids = input_ids,
        attention_mask = attention_mask,
        labels = labels
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar = True, logger = True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar = True, logger = True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar = True, logger = True)
    return loss

  def configure_optimizers(self):
#     return AdamW(self.parameters(), lr=3e-3)
    return AdamW(self.parameters(), lr=3e-5) 
#     return Adafactor(model.parameters(),
#                         lr=1e-3,
#                         eps=(1e-30, 1e-3),
#                         clip_threshold=1.0,
#                         decay_rate=-0.8,
#                         beta1=None,
#                         weight_decay=0.0,
#                         relative_step=False,
#                         scale_parameter=False,
#                         warmup_init=False
#                     )


In [ ]:
customModel = ClaimDecompModel()

## Load Fine-Tuned Model for Prediction

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

trainedModel2 = ClaimDecompModel.load_from_checkpoint("drive/MyDrive/past_checkpoints/V9L_B1_E7_SM850_TM500_LR3e5.ckpt")

trainedModel2.model.to(device)
trainedModel2.freeze()

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file drive/MyDrive/past_checkpoints/V9L_B1_E7_SM850_TM500_LR3e5.ckpt`


# Evaluate Fine-Tuned Model

## Check T5X Model Config Parameters

In [ ]:
 trainedModel2.model.config

T5Config {
  "_name_or_path": "google/t5-v1_1-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.27.4",
  "use_cache": true,
  "vocab_size": 32128
}

In [ ]:
grep MemTotal /proc/meminfo

# Running the T5X for question generation

In [ ]:
def predict(model, claim, justification):
  claim_encoding = tokenizer(
      claim,
      justification,
      max_length = 850,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  claim_encoding.to(device)

  prediction = model.model.generate(
      input_ids=claim_encoding["input_ids"],
      attention_mask=claim_encoding["attention_mask"],
      num_beams=1,
      max_length=800,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True,
      use_cache=True
  )

  decoded_res = [
      tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for pred in prediction
  ]
  
  if justification == "":
        justification = "No Justification provided"
        
  output = decoded_res[0].split("? ")
  print("Claim:\n\n" + claim + "\n\nJustification:\n\n" + justification + "\n\nGenerated Sub-Questions:\n")
  
  resultArr = []
  for i in range(0,len(output)):
        resp = str(i+1) + ": " + output[i] + "?"
        resultArr.append(resp)
  
  return " \n".join(resultArr)


def predictOrg(model, claim, justification):
  claim_encoding = tokenizer(
      claim,
      justification,
      max_length = 850,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  
  claim_encoding.to(device)

  prediction = model.generate(
      input_ids=claim_encoding["input_ids"],
      attention_mask=claim_encoding["attention_mask"],
      num_beams=1,
      max_length=800,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True,
      use_cache=True
  )

  decoded_res = [
      tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for pred in prediction
  ]
  
  output = decoded_res[0].split("? ")
  
  print("Claim:\n" + claim + "\nJustification:\n" + justification + "\nSub-Questions:\n")
  
  for i in range(0,len(output)):
        print(str(i) + ": " + output[i] + "?")
        

### Evaluating Our Fine-Tuned Model

In [ ]:
################################### Invalid Claims #######################################

## Lets try generating questions on song lyrics
lyrics = """Generate Questions: Welcome to Wonderland, I'll be your guide
Holding your hand under sapphire skies
Let's go exploring or we could just go for a walk
Welcome to Wonderland, where should we go
There's a tea party along down the road
Make an appearance and maybe they'll sing us a song"""

################################### Simple Claims #######################################

simpC1 = "Generate Questions: Donald Trump had a cup of coffee this morning and found that he was invited to the white house for the next election"

simpC2 = "Generate Questions: Putin has ordered the Russian invasion of Ukraine"

simpC3 = "Generate Questions: Boris Johnson just got elected as the new prime minister of the UK"

simpC4 = "Generate Questions: Former U.S. President Donald Trump signed checks for Christmas bonuses for executives at his namesake company as if they were independent contractors, longtime chief financial officer Allen Weisselberg testified on Thursday."

################################### General Claims #######################################

genClaim = "Generate Questions: Our CS3103 professor got COVID positive and conducted lecture over zoom instead of in-person classes"

## More about otters
otterClaim = "Generate Questions: The otter population has been rebounding since the 1970s when Singapore started cleaning up its waterways. Their numbers are rising partially because otters face no significant danger from other predators and only encounter crocodiles in a wetlands mangrove in the city’s north-west."

################################### Political Claims #######################################

polClaim = """Generate Questions: Says Sen. Jon Tester "believes that we should have a national registration so that the people of Montana should have to ask permission before they purchase a gun, ask permission from the federal government." """

polJustification = """Justification: Rosendale said Tester "believes that we should have a national registration so that the people of Montana should have to ask permission before they purchase a gun, ask permission from the federal government."
Tester’s record shows opposition to a federal list of lawfully-owned guns and gun owners. To claim he supports this is absurd.
We rate this Pants on Fire."""

manualQsn = "\nManual Decomposition: \n\nDoes Tester support the idea that the people of Montana ask for permission from the Federal Government to buy a gun?\nDoes this senator believe in having a national gun registration program?\nDoes Tester support gun purchases that are subject to federal approval?\nDoes this senator support gun registration programs in his own state?\n"


polClaim2 = "Generate Questions: A Facebook post stated on January 31, 2021: Nancy Pelosi bought $1.25 million in Tesla stock the day before Joe Biden signed an order “for all federal vehicles” to be electric."

polJustification2 = """Justification: An image shared on Facebook claims that Nancy Pelosi bought $1.25 million in Tesla stock the day before Biden
signed an order for all federal vehicles to be electric, implying that she sought to profit from inside information about new
government policies. The House speaker did report transactions involving Tesla stock, but the post misrepresented the purchases
and Biden’s policies to create the false impression that the transactions represented improper insider trading in Tesla shares. The statement contains an element of truth, but ignoring critical facts would give a different impression."""

polClaim3 = "When I withdrew in June of 2008, polls were showing that at least 40 percent of my supporters said, oh, they weren't going to support Sen. Obama."

polJustification3 = """Clinton said, "When I withdrew in June of 2008, polls were showing that at least 40 percent of my supporters said, oh, they weren't going to support Sen. Obama." The wording of polls makes exact comparisons a bit tricky, but in general, Clinton is right about the scale of Obama’s 2008 problem with dissident Democrats -- and correct that Obama had a harder road then than she seems to have now. We rate the statement Mostly True."""

test = """Generate Questions: Says Kentucky Rep. Andy Barr "would let shady payday lenders take advantage of our troops" and that he took "$36,550 from payday lenders."""

# predict(trainedModel, polClaim, polJustification)
# print(manualQsn)

# predict(trainedModel2, polClaim2, polJustification2)
predict(trainedModel2, test, "")
# predict(trainedModel2, otterClaim, "")
# predict(trainedModel, simpC4, "")



Claim:

Generate Questions: Says Kentucky Rep. Andy Barr "would let shady payday lenders take advantage of our troops" and that he took "$36,550 from payday lenders.

Justification:

No Justification provided

Generated Sub-Questions:

1: Did Barr allow payday lenders to take advantage of our troops?', "Did Barr allow payday lenders to take advantage of our troops?", 'Would Barr let payday lenders take advantage of our troops??


## Load required dataset for decomposing with T5x

In [ ]:
testPath = "./drive/MyDrive/filtered_data/evalDecompTest-qgen.csv"


dfTest = pd.read_csv(testPath)
dfTest.head()

## Predict sub-questions for the loaded dataset

In [ ]:
for i in tqdm(range(len(dfTest))):
  claim = dfTest.loc[i, 'claim']
  resp = predict(trainedModel2, claim, "")
  dfTest.loc[i, "t5-decomp-subquestions"] = resp

  0%|          | 1/200 [00:15<50:13, 15.14s/it]

Claim:

Says Kentucky Rep. Andy Barr "would let shady payday lenders take advantage of our troops" and that he took "$36,550 from payday lenders."

Justification:

No Justification provided

Generated Sub-Questions:



  1%|          | 2/200 [00:23<37:12, 11.27s/it]

Claim:

"New reports show Kevin Nicholson made over $1 million advising companies that shut down Wisconsin plants and laid off 1,900 people."

Justification:

No Justification provided

Generated Sub-Questions:



  2%|▏         | 3/200 [00:30<30:59,  9.44s/it]

...

Claim:

"In 2009 … Hillary Clinton was at the State Department working with U.S. corporations to pressure Haiti not to raise the minimum wage to 61 cents an hour from 24 cents."

Justification:

No Justification provided

Generated Sub-Questions:



100%|█████████▉| 199/200 [35:38<00:10, 10.56s/it]

Claim:

"The candidates controlled by Madigan, all of them have said, ‘The answer to our problems is a massive income tax hike.’ "

Justification:

No Justification provided

Generated Sub-Questions:



100%|██████████| 200/200 [35:52<00:00, 10.76s/it]

Claim:

"There’s actually a court order that prevents keeping the kids with the parents when you put the parents in jail."

Justification:

No Justification provided

Generated Sub-Questions:



In [ ]:
dfTest.head()

,claim,annotated-questions,davinci-decomp-subquestions,t5-decomp-subquestions,distinct-annotated-qsn,t5-cov,davinci-cov
0,"Says Kentucky Rep. Andy Barr ""would let shady ...","['1. Has Barr received $36,550 from payday len...",""" 1. Did Barr take money from these lenders?\n...",1: Did Barr allow payday lenders to take advan...,NaN,NaN,NaN
1,"""New reports show Kevin Nicholson made over $1...",['1. Did Nicholson make $1 million consulting ...,""" 1. Did Nicholson make over $1 million?\n2. D...",1: Did Nicholson advise companies that shut do...,NaN,NaN,NaN
2,Says that unless the recipient called back abo...,['1. Will people be taken into local police cu...,""" 1. Did someone call about this person's tax ...",1: Did the recipient call back??,NaN,NaN,NaN
3,"""Donald Trump said he was excited for the 2008...","['1. Was Donald Trump ""Excited"" for the 2008 h...",""" 1. Did Donald Trump say he was excited for t...",1: Did Trump say he was excited for the 2008 h...,NaN,NaN,NaN
4,"""The president has said the national security ...",['1. Has Obama cited climate change as the top...,""" 1. Did the president say that global warming...",1: Did Trump say the national security threat ...,NaN,NaN,NaN


In [ ]:
fileName = "./drive/MyDrive/filtered_data/final.csv"
dfTest.to_csv(fileName, index=False, encoding = 'utf-8-sig', header=True, )

### Evaluating with the original model

In [ ]:
version = "google/t5-v1_1-small" #"google/t5-v1_1-large" #"google/t5-v1_1-xl" is too huge for google colab to handle

ogmodel = AutoModelForSeq2SeqLM.from_pretrained(version)
ogtokenizer = AutoTokenizer.from_pretrained(version)

predictOrg(ogmodel, claim, justification)

### Evaluating individual predictions

In [ ]:
input_str = "Generate Questions: A Facebook post stated on January 31, 2021 Nancy Pelosi bought $1.25 million in Tesla stock the day before Joe Biden signed an order “for all federal vehicles” to be electric."
# input_str = "translate English to German: who is donald trump?"

encoded_input = tokenizer(input_str, padding=True, return_tensors="pt")
input_ids = encoded_input.input_ids
attention_mask = encoded_input.attention_mask

#The token id used to pad shorter sentences for batch training
#print(tokenizer.pad_token_id) 

# print(encoded_input)
# print(tokenizer.decode(encoded_input))

result = model.generate(input_ids = input_ids, attention_mask = attention_mask)

#To read the model response
decodedResult = [
    tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for gen_id in result
]

print("Response: " + decodedResult[0])
